In [4]:
import pandas as pd
import requests

In [5]:
csv_file = 'full_dataset.csv'
json_url = 'https://microdata.worldbank.org/index.php/api/tables/data/fcv/wld_2021_rtfp_v02_m'
limit = 99
country = 'Philippines'

In [6]:
def update_csv_from_json(csv_file, json_url, limit, country):
    # Load CSV data
    try:
        df_csv = pd.read_csv(csv_file)
    except FileNotFoundError:
        print(f"CSV file '{csv_file}' not found.")
        return

    # Calculate offset (based on the current length of the CSV)
    offset = len(df_csv)
    
    # Fetch new JSON data from the web using offset and limit
    try:
        response = requests.get(json_url, params={'limit': limit, 'offset': offset, 'country': country})
        response.raise_for_status()  # Check if the request was successful
        json_data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching JSON data: {e}")
        return
    
    # Extract the metadata and data from the JSON response
    found = json_data.get('metadata', {}).get('found', 0)
    data = json_data.get('data', [])

    # Check if there is new data to add
    if len(data) > 0:
        df_json = pd.json_normalize(data)
        df_csv = pd.concat([df_csv, df_json], ignore_index=True)

        # Save updated CSV file
        df_csv.to_csv(csv_file, index=False)
        print(f"CSV file updated with {len(df_json)} new rows. Total records: {found}")
    else:
        print("No new data to add.")


update_csv_from_json(csv_file, json_url, limit, country)


No new data to add.


In [10]:
df_csv = pd.read_csv(csv_file)
print(len(df_csv))

23653
